This example is based on the following studies:

> **Evaluating key controls on sediment flux to the Gulf of Corinth over the last 130 kyrs using a forward modeling approach** -- Sofia Pechlivanidou, Guillaume Duclaux, Tristan Salles, Casey Nixon, Patience Cowie, Robert Gawthorpe, Ritske Huismans 

+ AGU -- EP029:_'Mountain Peak to Sea Floor: Processes, Interactions and Feedbacks between Sediment Supply and Landscape Evolution'_, 2017 
+ EGU  -- TS6.1/GD7.4/GM4.7/GMPV8.6/SSP3.17:_'Evolution and architecture of rifts and passive margins: from mantle dynamics to surface processes'_, 2018. 

In [ ]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter(action = "ignore", category = FutureWarning)

import cmocean as cmo
from matplotlib import cm
from scripts import readInput as rInput

# display plots in SVG format
%config InlineBackend.figure_format = 'svg'
%matplotlib inline

# Evaluating key controls on erosion and deposition within active rifts


We use a numerical modeling approach to explore key controls on erosion and deposition within active rifts. 
 
<div class="alert alert-block alert-info">We will simulate the landscape development and basin stratigraphy for one of the most active rifts in the world, the **Corinth Rift**, over the past 130 kyrs.  We will see the different methods available in Badlands to constrain our model with field data and potentially assess the relative importance of: i) tectonic uplift/subsidence rate, ii) drainage area, iii) pre-existing topography, and iv) lithology on sediment delivered to the rift.</div>

***

## Study area

The Corinth Rift in central Greece is considered one of the most rapidly extending continental rifts on Earth that initiated <5 Ma (Ori, 1989; Leeder et al., 2008).

<img src="images/area.png" alt="geometry" width="340" height="400"/>

Northward migration of fault activity since $\sim$0.6 Ma and the progressive growth and linkage of predominantly north-dipping faults has resulted in the formation of an E - W oriented asym- metric marine graben, $\sim$120 km long and up to 30 km wide (_i.e._ the Gulf of Corinth).

The Gulf of Corinth records $\sim$2.5 km of offshore syn-rift deposits. Over the Late Quaternary sedimentation was characterized by alternating marine and lacustrine conditions, related to 100 kyr glacio-eustatic cycles, that produce characteristic seismic reflection profiles (_e.g._ Bell et al., 2009; Taylor et al., 2011; Nixon et al., 2016).


Isochore map highlighting the presence of a continuous depocenter developed as a responce to linkage of major north-dipping faults along the southern rift margin (Nixon et al., 2016).

<img src="images/volume.png" alt="geometry" width="600" height="400"/>

***

Let's have a look at the initial forcing conditions that will be use in this example.


## Initial surface 

In [ ]:
forcings = rInput.readInput()

In [ ]:
forcings.readDEM('data/gbrnodes.csv')

On all the plots below we will use the cmocean [colormap](https://matplotlib.org/cmocean/).

In [ ]:
forcings.plotInputGrid(data=forcings.Z,title="elevation",
                       mind=-4000.,maxd=4000,color=cmo.cm.delta)

***

## Sea level

Global sea-level curve is used to account for eustatic vs. lacustrine fluctuations of the base-level.

In [ ]:
forcings.readSea('data/lambeck14.csv') 

## Precipitation map 

The precipitation map has been extracted from present day annual rainfall regime using the following website **http://www.worldclim.org**.

In [ ]:
forcings.readRain('data/gbrrain.csv')

In [ ]:
forcings.plotInputGrid(data=forcings.rain,title="precipitation [m/y]",
                       mind=0.,maxd=4,color=cmo.cm.ice_r)

## Bedrock map

Mapped lithostratigraphic units are clustered based on elevation and slope and used to classify bedrock erodibility. In Sofia's model three groups are defined.

<img src="images/ero.png" alt="geometry" width="500" height="400"/>

In [ ]:
forcings.readEroLay('data/gbrbase.csv')

In this example, we modify the initial setting and chose 2 different layers:
+ **layer 1** is a bottom uniform layer -- erodibility: $\kappa = 5.e^{-6}$; thickness: 1000. m
+ **layer 2** is the surface layer with varying erodibility and thickness.


In the top layer (layer 2), we simplify the bedrock erodibility map propose in Sofia's experiment and define only 2 groups based on _elevation_:
+ **group 1**: erodibility ($\kappa = 3.e^{-6}$) above 700 m elevation with a thickness of 5. m
+ **group 2**: erodibility ($\kappa = 1.e^{-6}$) below 700 m elevation with a thickness of 10. m

We can visualise the top layer definition which is passed to Badlands:

In [ ]:
forcings.plotInputGrid(data=forcings.erolays[0],title="erodibility layer 2",
                       mind=1.e-6,maxd=5.e-6,color=cmo.cm.matter)

In [ ]:
forcings1 = rInput.readInput()

forcings1.readDEM('data/gbr500north.csv')
forcings1.plotInputGrid(data=forcings1.Z,title="elevation",
                       mind=-4000.,maxd=4000,color=cmo.cm.delta)

forcings1.readRain('data/rain500north.csv')
forcings1.plotInputGrid(data=forcings1.rain,title="precipitation [m/y]",
                       mind=0.,maxd=4,color=cmo.cm.ice_r)

forcings1.readEroLay('data/base500north.csv')
forcings1.plotInputGrid(data=forcings1.erolays[0],title="erodibility layer 2",
                       mind=1.e-6,maxd=5.e-6,color=cmo.cm.matter)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cmocean as cmo
from matplotlib import cm
from pylab import rcParams
from scipy.interpolate import RegularGridInterpolator
from mpl_toolkits.axes_grid1 import make_axes_locatable

xi = np.reshape(forcings1.X,(forcings1.nx, forcings1.ny),order='F')
yi = np.reshape(forcings1.Y,(forcings1.nx, forcings1.ny),order='F')
print xi.min(),xi.max()
print yi.min(),yi.max()

erange = [90000,500000,-1850000,-1480000]

r1,c1 = np.where(np.logical_and(xi>=erange[0],xi<=erange[1]))
r2,c2 = np.where(np.logical_and(yi>=erange[2],yi<=erange[3]))

rx = xi[r1.min():r1.max(),c2.min():c2.max()]
ry = yi[r1.min():r1.max(),c2.min():c2.max()]
zdata = forcings1.Z[r1.min():r1.max(),c2.min():c2.max()]
rdata = forcings1.rain[r1.min():r1.max(),c2.min():c2.max()]
bdata = forcings1.erolays[0][r1.min():r1.max(),c2.min():c2.max()]
print rdata.shape,forcings1.Z.shape,bdata.shape
print rx.shape

In [ ]:
xgrid = np.arange(0,rx.shape[0]*forcings.dx,forcings.dx)
ygrid = np.arange(0,rx.shape[1]*forcings.dx,forcings.dx)
RGI_function1 = RegularGridInterpolator((xgrid, ygrid), zdata)
RGI_function2 = RegularGridInterpolator((xgrid, ygrid), rdata)
RGI_function3 = RegularGridInterpolator((xgrid, ygrid), bdata)

res = forcings.dx*2.
ngridX = np.arange(0,xgrid.max(),res)
ngridY = np.arange(0,ygrid.max()+res,res)
xii,yii = np.meshgrid(ngridX, ngridY)

newz = RGI_function1((xii.flatten(),yii.flatten()))
newr = RGI_function2((xii.flatten(),yii.flatten()))
newb = RGI_function3((xii.flatten(),yii.flatten()))

df = pd.DataFrame({'X':xii.flatten(),'Y':yii.flatten(),'Z':newz.flatten()})
df.to_csv('data/gbrnodes.csv',columns=['X', 'Y', 'Z'], sep=' ', index=False ,header=0)


df = pd.DataFrame({'X':xii.flatten(),'Y':yii.flatten(),'Z':newr.flatten()})
df.to_csv('data/gbrrain.csv',columns=['Z'], sep=' ', index=False ,header=0)


newb[newb<1]=0
df = pd.DataFrame({'X':xii.flatten(),'Y':yii.flatten(),'Z':newb.flatten()})
df.to_csv('data/gbrbase.csv',columns=['Z'], sep=' ', index=False ,header=0)

In [ ]:
forcings2 = rInput.readInput()

forcings2.readDEM('data/gbrnodes.csv')
forcings2.plotInputGrid(data=forcings2.Z,title="elevation",
                       mind=-4000.,maxd=4000,color=cmo.cm.delta)

forcings2.readRain('data/gbrrain.csv')
forcings2.plotInputGrid(data=forcings2.rain,title="precipitation [m/y]",
                       mind=0.,maxd=4,color=cmo.cm.ice_r)

forcings2.readEroLay('data/gbrbase.csv')
forcings2.plotInputGrid(data=forcings2.erolays[0],title="erodibility layer 2",
                       mind=1.e-6,maxd=5.e-6,color=cmo.cm.matter)